In [1]:
# @title genera pokemon.csv
import urllib.request
import pandas as pd
url = 'https://pokemondb.net/pokedex/all'
# Imposta un header User-Agent per simulare una richiesta da browser
req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
with urllib.request.urlopen(req) as response:
    html = response.read()
# Usa pandas per estrarre la tabella dalla stringa HTML
tables = pd.read_html(html)
df = tables[0]
# Salva il dataframe in un file CSV
df.to_csv('pokemons.csv', index=False)

In [ ]:
# @title genera moves.csv
import requests
from bs4 import BeautifulSoup
import csv

url = 'https://pokemondb.net/move/all'
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
# Trova la tabella dei movimenti
table = soup.find('table', class_='data-table')
rows = table.find_all('tr')
# Estrae le intestazioni della tabella
header_row = rows[0]
headers_list = [th.get_text(strip=True) for th in header_row.find_all('th')]
# Scrive il CSV con i dati
with open('moves.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers_list)
    for row in rows[1:]:
        cols = row.find_all('td')
        row_data = []
        for i, col in enumerate(cols):
            # Se la colonna è la categoria (controlla l'intestazione)
            if headers_list[i] in ['Cat.', 'Category']:
                img = col.find('img')
                if img and img.has_attr('alt'):
                    row_data.append(img['alt'])
                else:
                    row_data.append('')
            else:
                row_data.append(col.get_text(strip=True))
        writer.writerow(row_data)

print("Il file 'moves.csv' è stato generato correttamente.")

Il file 'moves.csv' è stato generato correttamente.


In [3]:
# @title genera type_pairs.csv
import urllib.request
import pandas as pd
# Recupera la pagina e salva il CSV originale
url = 'https://pokemondb.net/type/dual'
req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
with urllib.request.urlopen(req) as response:
    html = response.read()

tables = pd.read_html(html)
df = tables[0]
# Salva il CSV originale (che contiene la colonna "PKMN")
df.to_csv('type.csv', index=False)

# Rileggi il CSV impostando la prima colonna ("PKMN") come indice
# In questo CSV, la colonna "PKMN" rappresenta il tipo difensivo
df = pd.read_csv('type.csv', index_col=0)

# Crea una lista di record invertendo i ruoli:
# La colonna (header) diventa il tipo attaccante
# L'indice (riga) diventa il tipo difensivo
rows_list = []
for defensive_type, row in df.iterrows():
    for attacking_type, cell in row.items():
        # Salta le righe se "PKMN" appare in uno dei tipi (ignorando case e spazi)
        if "PKMN" in attacking_type.strip().upper() or "PKMN" in defensive_type.strip().upper():
            continue

        # Controlla se la cella è vuota o NaN
        if pd.isna(cell):
            multiplier_num = 1.0
        else:
            multiplier_str = str(cell).strip()
            # Se il valore è vuoto o contiene "—", usa 1.0
            if multiplier_str == "" or multiplier_str == "—":
                multiplier_num = 1.0
            elif multiplier_str in ["½", "1/2"]:
                multiplier_num = 0.5
            elif multiplier_str in ["¼", "1/4"]:
                multiplier_num = 0.25
            else:
                try:
                    multiplier_num = float(multiplier_str)
                except ValueError:
                    multiplier_num = 1.0  # fallback a 1.0 se la conversione fallisce
        # Invertiamo: il tipo attaccante è la colonna, il difensivo è l'indice
        rows_list.append({
            'attacking_type': attacking_type,
            'defending_type': defensive_type,
            'multiplier': multiplier_num
        })
# Crea un nuovo DataFrame con le coppie e salva il risultato in un nuovo file CSV
pairs_df = pd.DataFrame(rows_list)
pairs_df.to_csv('types_pairs.csv', index=False)
print("Il file 'types_pairs.csv' è stato generato correttamente.")

Il file 'types_pairs.csv' è stato generato correttamente.


In [ ]:
# @title genera pokemon.sql
import csv
# Nome del file CSV da cui leggere i dati e del file SQL che verrà generato
csv_file = 'pokemons.csv'
sql_file = 'pokemons_inserts.sql'
with open(csv_file, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    data = list(reader)

with open(sql_file, 'w', encoding='utf-8') as f:
    # Creazione della tabella: il CSV contiene anche la colonna Total
    f.write("DROP TABLE IF EXISTS pokemon;\n")
    f.write("CREATE TABLE pokemon (\n")
    f.write("    Name TEXT,\n")
    f.write("    Type1 TEXT,\n")
    f.write("    Type2 TEXT,\n")
    f.write("    Total INTEGER,\n")
    f.write("    HP INTEGER,\n")
    f.write("    Attack INTEGER,\n")
    f.write("    Defense INTEGER,\n")
    f.write("    \"Sp. Atk\" INTEGER,\n")
    f.write("    \"Sp. Def\" INTEGER,\n")
    f.write("    Speed INTEGER\n")
    f.write(");\n\n")

    # Genera i comandi INSERT per ogni riga del CSV
    for row in data:
        # Gestione del nome: escape degli apostrofi
        name = row['Name'].replace("'", "''") if row.get('Name') else ''

        # La colonna "Type" contiene i tipi separati da spazi (es. "Grass  Poison")
        type_field = row['Type'].strip() if row.get('Type') else ''
        tokens = type_field.split()
        if len(tokens) == 0:
            type1 = ''
            type2 = "NULL"
        elif len(tokens) == 1:
            type1 = tokens[0].replace("'", "''")
            type2 = "NULL"
        else:
            type1 = tokens[0].replace("'", "''")
            type2 = "'" + tokens[1].replace("'", "''") + "'"

        # Gestione dei campi numerici: se vuoti, inserisce NULL
        total   = row['Total'].strip() if row.get('Total') and row['Total'].strip() != "" else 'NULL'
        hp      = row['HP'].strip() if row.get('HP') and row['HP'].strip() != "" else 'NULL'
        attack  = row['Attack'].strip() if row.get('Attack') and row['Attack'].strip() != "" else 'NULL'
        defense = row['Defense'].strip() if row.get('Defense') and row['Defense'].strip() != "" else 'NULL'
        sp_atk  = row['Sp. Atk'].strip() if row.get('Sp. Atk') and row['Sp. Atk'].strip() != "" else 'NULL'
        sp_def  = row['Sp. Def'].strip() if row.get('Sp. Def') and row['Sp. Def'].strip() != "" else 'NULL'
        speed   = row['Speed'].strip() if row.get('Speed') and row['Speed'].strip() != "" else 'NULL'

        # Costruzione della query INSERT
        sql = (
            "INSERT INTO pokemon (Name, Type1, Type2, Total, HP, Attack, Defense, \"Sp. Atk\", \"Sp. Def\", Speed) VALUES ("
            f"'{name}', '{type1}', {type2}, {total}, {hp}, {attack}, {defense}, {sp_atk}, {sp_def}, {speed}"
            ");\n"
        )
        f.write(sql)

print(f"Il file SQL '{sql_file}' è stato generato correttamente.")

Il file SQL 'pokemons_inserts.sql' è stato generato correttamente.


In [5]:
# @title genera moves.sql
# Nome del file CSV da cui leggere i dati e del file SQL da generare
csv_file = 'moves.csv'
sql_file = 'moves_inserts.sql'
# Funzione per processare i campi numerici: se il valore è vuoto, "—", "infinito" o "∞", restituisce NULL
def process_numeric(value):
    value = value.strip()
    if value == "" or value == "—" or value.lower() == "infinito" or value == "∞":
        return "NULL"
    else:
        return value
# Funzione per processare i campi testuali: se vuoto o "—", restituisce NULL, altrimenti effettua l'escape delle virgolette singole
def process_text(value):
    value = value.strip()
    if value == "" or value == "—":
        return "NULL"
    else:
        return "'" + value.replace("'", "''") + "'"
# Legge i dati dal CSV
with open(csv_file, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    rows = list(reader)
with open(sql_file, 'w', encoding='utf-8') as f:
    # Scrive i comandi per eliminare la tabella se esiste e poi crearla
    f.write("DROP TABLE IF EXISTS moves;\n")
    f.write("CREATE TABLE moves (\n")
    f.write("    name TEXT,\n")
    f.write("    type TEXT,\n")
    f.write("    category TEXT,\n")
    f.write("    power INTEGER,\n")
    f.write("    acc INTEGER,\n")
    f.write("    pp INTEGER,\n")
    f.write("    effect TEXT,\n")
    f.write("    prob INTEGER\n")
    f.write(");\n\n")

    # Genera i comandi INSERT per ogni riga del CSV
    for row in rows:
        name     = process_text(row['Name'])
        type_val = process_text(row['Type'])
        category = process_text(row['Cat.'])
        power    = process_numeric(row['Power'])
        acc      = process_numeric(row['Acc.'])
        pp       = process_numeric(row['PP'])
        effect   = process_text(row['Effect'])
        prob     = process_numeric(row['Prob. (%)'])

        insert_cmd = (
            "INSERT INTO moves (name, type, category, power, acc, pp, effect, prob) VALUES ("
            f"{name}, {type_val}, {category}, {power}, {acc}, {pp}, {effect}, {prob}"
            ");\n"
        )
        f.write(insert_cmd)

print(f"Il file SQL '{sql_file}' è stato generato correttamente.")

Il file SQL 'moves_inserts.sql' è stato generato correttamente.


In [6]:
# @title genera types.sql
# Nome dei file di input e output
csv_file = 'types_pairs.csv'
sql_file = 'types_inserts.sql'
# Dizionario per il mapping delle abbreviazioni ai nomi per esteso
type_mapping = {
    "Bug": "Bug",
    "Dar": "Dark",
    "Dra": "Dragon",
    "Ele": "Electric",
    "Fai": "Fairy",
    "Fig": "Fighting",
    "Fir": "Fire",
    "Flying": "Flying",
    "Fly": "Flying",
    "Gho": "Ghost",
    "Gra": "Grass",
    "Gro": "Ground",
    "Ground": "Ground",
    "Ice": "Ice",
    "Nor": "Normal",
    "Normal": "Normal",
    "Poi": "Poison",
    "Psy": "Psychic",
    "Roc": "Rock",
    "Ste": "Steel",
    "Steel": "Steel",
    "Wat": "Water"
}

def get_full_type(abbr):
    # Se il tipo contiene "—", rimuovi tutto dopo (e incluso) "—"
    if "—" in abbr:
        abbr = abbr.split("—")[0].strip()
    else:
        abbr = abbr.strip()
    return type_mapping.get(abbr, abbr)

# Apri il CSV in input e il file SQL in output
with open(csv_file, newline='', encoding='utf-8') as csvfile, open(sql_file, 'w', encoding='utf-8') as sqlfile:
    reader = csv.DictReader(csvfile)

    # Scrive il comando per eliminare la tabella se esiste ed crearla
    sqlfile.write("DROP TABLE IF EXISTS type_effectiveness;\n")
    sqlfile.write("CREATE TABLE type_effectiveness (\n")
    sqlfile.write("    attacking_type TEXT,\n")
    sqlfile.write("    defending_type TEXT,\n")
    sqlfile.write("    multiplier REAL\n")
    sqlfile.write(");\n\n")

    # Genera le INSERT applicando il mapping e filtrando le righe non desiderate
    for row in reader:
        # Legge i valori dal CSV
        att = row['attacking_type']
        defn = row['defending_type']
        multiplier = row['multiplier'].strip()

        # Salta le righe dove compare l'header (es. "ATTACK →  DEFENSE ↴")
        if att.strip().upper() == "ATTACK →  DEFENSE ↴" or defn.strip().upper() == "ATTACK →  DEFENSE ↴":
            continue

        # Applica il mapping a entrambi i tipi
        attacking = get_full_type(att)
        defending = get_full_type(defn)

        # Non serve modificare il moltiplicatore (già impostato nel CSV)
        # Effettua l'escape degli apostrofi
        attacking_esc = attacking.replace("'", "''")
        defending_esc = defending.replace("'", "''")

        sqlfile.write(
            f"INSERT INTO type_effectiveness (attacking_type, defending_type, multiplier) VALUES ('{attacking_esc}', '{defending_esc}', {multiplier});\n"
        )

print(f"Il file SQL '{sql_file}' è stato generato correttamente.")

Il file SQL 'types_inserts.sql' è stato generato correttamente.
